In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import os
from urllib.request import urlopen

In [ ]:
spark = SparkSession.builder \
        .master('local[*]') \
        .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
        .config("spark.sql.warehouse.dir","/users/hive/warehouse/") \
        .appName('myappname') \
        .enableHiveSupport() \
        .getOrCreate()

In [ ]:
#Default Read By Pandas
df = pd.read_json("https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces")

## อ่านไฟล์ JSON ด้วย Pyspark 

In [ ]:
#แบบที่ 1
url = 'https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces'
jsonData = urlopen(url).read().decode('utf-8')
rdd = spark.sparkContext.parallelize([jsonData])
df = spark.read.json(rdd)
df.select(['year']).limit(5).toPandas()
#แบบที่ 2
url = "https://covid19.ddc.moph.go.th/api/Cases/round-1to2-line-lists"
r = requests.get(url)
df = spark.createDataFrame(r.json()['data'], schema=schema)

## สร้าง Dataframe ด้วย Pyspark

In [ ]:
mydf = spark.createDataFrame(df)
#Simple View
mydf.collect()
#Dataframe view
#mydf.show()

## อ่าน CSV ด้วย Pyspark

In [ ]:
mydf3 = spark.read.csv("path/csv.csv", header=True, inferSchema=True)
mydf3.limit(10).toPandas()

## ปริ้นดู Schema ด้วย Pyspark

In [ ]:
mydf.printSchema()

## Drop Columns

In [ ]:
mydf = mydf.drop('year')

## เลือกข้อมูลแบบ Select

In [ ]:
mydf.select(['province', 'update_date','new_case']).limit(10).toPandas()

## Filter เลือกข้อมูล Where

In [ ]:
mydf.filter(mydf.province == 'ปัตตานี').limit(10).toPandas()

## Group by

In [ ]:
mydf.groupby('province').count().limit(10).toPandas()

## สร้าง View Table

In [ ]:
mydf.createOrReplaceTempView('covid_temp')
spark.sql('SELECT * FROM covid_temp').limit(10).toPandas()

## เลือกข้อมูลแบบหลายเงื่อนไข

In [ ]:
mydf.select(['province','update_date', 'new_case', 'total_case']) \
    .filter(mydf.province == 'ปัตตานี') \
    .limit(10) \
    .toPandas()

## สร้าง Column ใม่และสร้างแบบมีเงือนไข

In [ ]:
mydf =  mydf.withColumn("x4", lit(None)).toPandas() #date_add(col('col'), 3) บวกวันที่

## แทนที่ค่าว่าง

In [ ]:
mydf.na.fill('Missing Value', 'privince').show()

# เช็คว่ามี Table ยัง

In [ ]:
if (spark.sql("show tables in pyspark")
         .filter(col("tableName") == "plants")
         .count() > 0):
    print('found')   
else:
    print('not found')